In [2]:
from db.fetch import fetch_table


fetch_table("news_articles")

[(1,
  1,
  'Heavy rains expected to boost umbrella sales in Mumbai',
  'Analysts suggest seasonal demand will drive short-term stock gains for umbrella companies.',
  'https://dummynews.com/umbrella-sales',
  None,
  3,
  0.95,
  datetime.datetime(2025, 10, 4, 1, 58, 7, 164218))]

In [1]:
from langgraph.graph import StateGraph, START, END, MessagesState
from db.insertion import insert_dummy_article
from agents.categorizer import categorize_node


class State(MessagesState):
    article_id: int
    title: str
    content: str
    summary: str = ""  # optional field


if __name__ == "__main__":

    # Step 1: insert dummy article
    article_id = insert_dummy_article()
    # article_id = 401

    # Step 2: build LangGraph workflow
    graph = StateGraph(State)
    graph.add_node("categorize", categorize_node)
    graph.add_edge(START, "categorize")
    graph.add_edge("categorize", END)

    graph = graph.compile()

    # Step 3: run categorization
    state = State(
        article_id=article_id,
        title="Heavy rains expected to boost umbrella sales in Mumbai",
        content="Analysts suggest seasonal demand will drive short-term stock gains for umbrella companies."
    )

    result = graph.invoke(state, start="categorize")
    print("✅ Categorization Result:", result)


c:\Users\91935\vaish\News-Trust-Agent\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ Dummy article inserted with ID 1
----------------LLM RESPONSE----------:  ```json
{
"category": "Seasonal",
"confidence": 0.95
}
```
----------------article_id----------:  1
----------------LLM category----------:  Seasonal
----------------LLM confidence----------:  0.95
(3,)
3
Seasonal
0.95
✅ Categorization Result: {'messages': [], 'article_id': 1, 'title': 'Heavy rains expected to boost umbrella sales in Mumbai', 'content': 'Analysts suggest seasonal demand will drive short-term stock gains for umbrella companies.'}


Trace(trace_id=tr-51ca0682d5d40e62a72a525ebed20c48)

In [10]:
from db.insertion import save_category

save_category(101,"Seasonal", 0.23)

In [1]:
from agents.categorizer import categorize_node
categorize_node()

2025-10-03 17:12:39,320 - INFO - Using direct username/password for MLflow authentication
2025-10-03 17:12:41,014 - INFO - Successfully retrieved auth token


----------------LLM RESPONSE----------:  {
  "category": "Seasonal",
  "confidence": 0.95
}
----------------article_id----------:  101
----------------LLM category----------:  Seasonal
----------------LLM confidence----------:  0.95
(3,)
3
Seasonal
0.95


{'category': 'Seasonal', 'confidence': 0.95}

Trace(trace_id=tr-89d77065c5f7e6f163a101ff5ac6d1ec)

In [1]:
from mlflow_client import mlflow_client
mlflow_client()


In [6]:
import mlflow
import mlflow.sklearn
from sklearn.linear_model import LinearRegression
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from mlflow_client import mlflow_client

mlflow_client()

mlflow.enable_system_metrics_logging()

mlflow.sklearn.autolog()

# Generate toy dataset
X, y = make_regression(n_samples=100, n_features=2, noise=0.1, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model
model = LinearRegression()
model.fit(X_train, y_train)

# Evaluate
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)

# mlflow.log_metric("mse", mse)


# with mlflow.start_run():

#     # Log parameters
#     mlflow.log_param("fit_intercept", model.fit_intercept)

#     # Log metric
#     mlflow.log_metric("mse", mse)

#     # Log the model
#     mlflow.sklearn.log_model(model, artifact_path="model")

#     print("Run ID:", mlflow.active_run().info.run_id)

# mlflow.end_run()



2025/10/03 19:37:05 WARNING mlflow.utils.autologging_utils: MLflow sklearn autologging is known to be compatible with 1.3.1 <= scikit-learn <= 1.7.1, but the installed version is 1.7.2. If you encounter errors during autologging, try upgrading / downgrading scikit-learn to a compatible version, or try upgrading MLflow.
2025/10/03 19:37:05 INFO mlflow.system_metrics.system_metrics_monitor: Skip logging GPU metrics. Set logger level to DEBUG for more details.
2025/10/03 19:37:05 INFO mlflow.system_metrics.system_metrics_monitor: Started monitoring system metrics.
2025/10/03 19:37:05 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'b4413be9867e401ea3154234f3114006', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow
2025/10/03 19:37:13 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version speci

🏃 View run grandiose-trout-757 at: http://127.0.0.1:5000/#/experiments/926232657939640394/runs/b4413be9867e401ea3154234f3114006
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/926232657939640394


In [5]:
mlflow.end_run()

🏃 View run gifted-asp-497 at: http://127.0.0.1:5000/#/experiments/926232657939640394/runs/3647338ba5794fef98edbfe323639b31
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/926232657939640394
